In [ ]:
!pip install netCDF4
!pip install cftime

#il faut l'executer une fois pour importer les librairies

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 42.3 MB/s eta 0:00:00


In [ ]:
import xarray as xr
import numpy as np
import os
import cftime
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path='drive/MyDrive/projet/data'

In [ ]:
# Étape 1: Réduire la granularité des données
def reduce_granularity(ds, target_points=1000):
    lat_points = ds.sizes['lat']
    lon_points = ds.sizes['lon']
    total_points = lat_points * lon_points
    factor = int(np.sqrt(total_points / target_points))
    ds_reduced = ds.coarsen(lat=factor, lon=factor, boundary='trim').mean()
    return ds_reduced

# Conversion des objets cftime en datetime
def convert_cftime_to_datetime(time_data):
    if isinstance(time_data[0], cftime.datetime):
        return np.array([pd.Timestamp(date.isoformat()) for date in time_data])
    return pd.to_datetime(time_data)

# Étape 2: Séparer les données entre passées et présentes
def split_past_present(ds, past_end='1950-12-31', present_start='1951-01-01'):
    # Get the actual time range of the dataset
    start_date = ds['time'].min().dt.strftime('%Y-%m-%d').values.item()
    end_date = ds['time'].max().dt.strftime('%Y-%m-%d').values.item()
    # Adjust past_end if it's beyond the dataset's end date
    past_end = min(past_end, end_date)
    # Select data within the dataset's time range
    y_past = ds.sel(time=slice(start_date, past_end))
    y_present = ds.sel(time=slice(present_start, end_date))
    return y_past, y_present

# Étape 3: Calculer la saisonnalité (moyenne mensuelle)
def calculate_seasonalities(y_past):
    # Check if y_past is empty and handle it
    if y_past.time.size == 0:
        # If empty, return NaN or a suitable placeholder
        return None  # or np.nan, or a placeholder dataset

    seasonalities = y_past.groupby('time.month').mean('time')
    return seasonalities

# Étape 4: Calculer les anomalies
def calculate_anomalies(data, seasonalities):
    # Check if data or seasonalities are empty
    if data.time.size == 0 or seasonalities is None:
        # If empty, return NaN or a suitable placeholder
        return None  # or np.nan, or a placeholder dataset

    anomalies = data.groupby('time.month') - seasonalities
    return anomalies



# Étape 5: Stocker les anomalies dans des arrays numpy
def process_simulations(path):
    Y_past_anomalies = []
    Y_present_anomalies = []

    for file_name in os.listdir(path):
        file_path = os.path.join(path, file_name)
        if file_name.endswith('.nc'):
            ds = xr.open_dataset(file_path, decode_times=False)
            ds['time'] = convert_cftime_to_datetime(ds['time'].values)
            print(f"Processing file: {file_name}, time range: {ds['time'].min().values} to {ds['time'].max().values}")
            ds = reduce_granularity(ds)

            y_past, y_present = split_past_present(ds)

            seasonalities = calculate_seasonalities(y_past)

            y_past_anomalies = calculate_anomalies(y_past, seasonalities)
            y_present_anomalies = calculate_anomalies(y_present, seasonalities)

            Y_past_anomalies.append(y_past_anomalies)
            Y_present_anomalies.append(y_present_anomalies)

    return Y_past_anomalies, Y_present_anomalies

# Étape 6: Remodeler les dimensions
#def reshape_anomalies(Y_anomalies):
 #   reshaped_anomalies = np.vstack(
       # [anomaly.values.reshape(anomaly.shape[0] * anomaly.shape[1], -1) for anomaly in Y_anomalies]
    #)
    #return reshaped_anomalies

# Étape 7: Calculer la matrice de covariance C_N
def compute_covariance_and_signal(Y_past_reshaped, Y_present_reshaped):
    C_N = np.cov(Y_past_reshaped, rowvar=False)
    response_past = np.mean(Y_past_reshaped, axis=0)
    response_present = np.mean(Y_present_reshaped, axis=0)
    X = response_present - response_past
    return C_N, X

# Étape 8: Calculer le détecteur optimal beta
def calculate_beta(C_N, X):
    return np.linalg.solve(C_N, X)


Y_past_anomalies, Y_present_anomalies = process_simulations(path)

# Étape 2 : Reshaper les anomalies
#Y_past_reshaped = reshape_anomalies(Y_past_anomalies)
#Y_present_reshaped = reshape_anomalies(Y_present_anomalies)

# Étape 3 : Calculer la matrice de covariance et le signal
C_N, X = compute_covariance_and_signal(Y_past_anomalies, Y_present_anomalies)

# Étape 4 : Calculer le détecteur optimal beta
beta = calculate_beta(C_N, X)

print("Beta:", beta)


Processing file: tas_mon_CESM2_historical_ssp370_r1011.001i1p1f1.188001-202212.nc, time range: 1970-01-01T00:00:00.000010965 to 1970-01-01T00:00:00.000063129
Processing file: tas_mon_CESM2_historical_ssp370_r1031.002i1p1f1.188001-202212.nc, time range: 1970-01-01T00:00:00.000010965 to 1970-01-01T00:00:00.000063129
Processing file: tas_mon_CESM2_historical_ssp370_r1051.003i1p1f1.188001-202212.nc, time range: 1970-01-01T00:00:00.000010965 to 1970-01-01T00:00:00.000063129
Processing file: tas_mon_CESM2_historical_ssp370_r1071.004i1p1f1.188001-202212.nc, time range: 1970-01-01T00:00:00.000010965 to 1970-01-01T00:00:00.000063129
Processing file: tas_mon_CESM2_historical_ssp370_r1091.005i1p1f1.188001-202212.nc, time range: 1970-01-01T00:00:00.000010965 to 1970-01-01T00:00:00.000063129
Processing file: tas_mon_CESM2_historical_ssp370_r1111.006i1p1f1.188001-202212.nc, time range: 1970-01-01T00:00:00.000010965 to 1970-01-01T00:00:00.000063129
Processing file: tas_mon_CESM2_historical_ssp370_r11

TypeError: unsupported operand type(s) for +: 'NoneType' and 'NoneType'